In [19]:
import os
import pandas as pd
import numpy as np
import datetime
from datetime import datetime

data_dir = os.getcwd().replace("/notebooks","/data/interim")
csv_name = "combined_accounts.csv"
accts_path = os.path.join(data_dir, csv_name)

In [20]:
accts_df = pd.read_csv(accts_path)
accts_df.head()

,Source,date,Description,credit_flg,debit_flg,Amount,meta_category,category,sub_category,event
0,USAA CC,10/27/18,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-50.72,Discretionary,Alcohol,NaN,NaN
1,USAA CC,9/29/18,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-48.56,Discretionary,Alcohol,NaN,NaN
2,USAA CC,9/22/18,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-61.10,Discretionary,Alcohol,NaN,NaN
3,USAA CC,9/10/18,HIGHLAND WINE AND LIQUOR DENVER CO,False,True,-11.87,Discretionary,Alcohol,NaN,NaN
4,USAA CC,9/1/18,TOWN LIQUOR 0848700216503DENVER CO,False,True,-22.68,Discretionary,Alcohol,NaN,NaN


In [21]:
accts_df["date"] = pd.to_datetime(accts_df["date"])
accts_df["year"] = accts_df.apply(lambda x : x["date"].year, axis=1)
accts_df["month"] = accts_df.apply(lambda x : x["date"].month, axis=1)
accts_df["day"] = accts_df.apply(lambda x : x["date"].day, axis=1)

In [22]:
accts_df.head()

,Source,date,Description,credit_flg,debit_flg,Amount,meta_category,category,sub_category,event,year,month,day
0,USAA CC,2018-10-27,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-50.72,Discretionary,Alcohol,NaN,NaN,2018,10,27
1,USAA CC,2018-09-29,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-48.56,Discretionary,Alcohol,NaN,NaN,2018,9,29
2,USAA CC,2018-09-22,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-61.10,Discretionary,Alcohol,NaN,NaN,2018,9,22
3,USAA CC,2018-09-10,HIGHLAND WINE AND LIQUOR DENVER CO,False,True,-11.87,Discretionary,Alcohol,NaN,NaN,2018,9,10
4,USAA CC,2018-09-01,TOWN LIQUOR 0848700216503DENVER CO,False,True,-22.68,Discretionary,Alcohol,NaN,NaN,2018,9,1


In [23]:
accts_df.meta_category.unique()

array(['Discretionary', 'Obligatory', 'Required', 'Debt Payment',
       'Health', 'Salary', nan], dtype=object)

In [24]:
debits_df = accts_df[(accts_df.debit_flg == True) & (accts_df.year == 2018)]
debits_df = debits_df[debits_df.category != "Credit Card Payment"]
#debits_df = debits_df[debits_df.meta_category != "Savings"]
debits_by_metacat_df = debits_df.groupby(['month', 'meta_category'])['Amount'].sum().reset_index()
debits_by_metacat_df

,month,meta_category,Amount
0,5,Discretionary,-3474.01
1,5,Health,-140.00
2,5,Obligatory,-6.48
3,5,Required,-1852.81
4,6,Discretionary,-1919.59
5,6,Health,-229.98
6,6,Required,-1544.52
7,7,Debt Payment,-2000.00
8,7,Discretionary,-2418.01
9,7,Health,-136.00


In [25]:
debits_df[debits_df.month == 10].to_csv("/Users/gbolla/Desktop/october.csv")
debits_by_metacat_df.to_csv("/Users/gbolla/Desktop/category_spending.csv")

In [26]:
credits_df = accts_df[accts_df.credit_flg == True]
credits_df[credits_df.month == 10].to_csv("/Users/gbolla/Desktop/october_credits.csv")

In [27]:
monthly_salary = credits_df.groupby(['month', 'meta_category'])['Amount'].sum().reset_index()["Amount"].mean()

In [28]:
debits_by_metacat_df.meta_category.unique()

array(['Discretionary', 'Health', 'Obligatory', 'Required',
       'Debt Payment'], dtype=object)

In [29]:
print("discretionary spend mean: {}".format(discretionary_spend_mean))
print("required spend mean: {}".format(required_spend_mean))
print("health spend mean: {}".format(health_spend_mean))
print("obligatory spend mean: {}".format(obligatory_spend_mean))
print("debt payment spend mean: {}".format(debt_payment_spend_mean))

discretionary spend mean: -2883.2400000000002
required spend mean: -1795.1849999999997
health spend mean: -157.33
obligatory spend mean: -175.30666666666664
debt payment spend mean: -333.3333333333333


In [30]:
len(debits_by_metacat_df.month.unique())

6

In [31]:
num_months = float(len(debits_by_metacat_df.month.unique()))
discretionary_spend_mean = float(debits_by_metacat_df["Amount"][debits_by_metacat_df["meta_category"] == "Discretionary"].sum()) / num_months
required_spend_mean = float(debits_by_metacat_df["Amount"][debits_by_metacat_df["meta_category"] == "Required"].sum()) / num_months
health_spend_mean = float(debits_by_metacat_df["Amount"][debits_by_metacat_df["meta_category"] == "Health"].sum()) / num_months
obligatory_spend_mean = float(debits_by_metacat_df["Amount"][debits_by_metacat_df["meta_category"] == "Obligatory"].sum()) / num_months
debt_payment_spend_mean = float(debits_by_metacat_df["Amount"][debits_by_metacat_df["meta_category"] == "Debt Payment"].sum()) / num_months

print("discretionary spend mean: {}".format(discretionary_spend_mean))
print("required spend mean: {}".format(required_spend_mean))
print("health spend mean: {}".format(health_spend_mean))
print("obligatory spend mean: {}".format(obligatory_spend_mean))
print("debt payment spend mean: {}".format(debt_payment_spend_mean))

discretionary spend mean: -2883.2400000000002
required spend mean: -1795.1849999999997
health spend mean: -157.33
obligatory spend mean: -175.30666666666664
debt payment spend mean: -333.3333333333333


In [32]:
debits_by_metacat_df[debits_by_metacat_df.meta_category == "Obligatory"]

,month,meta_category,Amount
2,5,Obligatory,-6.48
10,7,Obligatory,-970.23
14,8,Obligatory,-17.29
18,9,Obligatory,-57.84


In [33]:
discretionary_df = debits_df[debits_df.meta_category == "Discretionary"]
discretionary_df.head()

,Source,date,Description,credit_flg,debit_flg,Amount,meta_category,category,sub_category,event,year,month,day
0,USAA CC,2018-10-27,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-50.72,Discretionary,Alcohol,NaN,NaN,2018,10,27
1,USAA CC,2018-09-29,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-48.56,Discretionary,Alcohol,NaN,NaN,2018,9,29
2,USAA CC,2018-09-22,ARGONAUT WINE &amp; LIQUOR S DENVER CO,False,True,-61.10,Discretionary,Alcohol,NaN,NaN,2018,9,22
3,USAA CC,2018-09-10,HIGHLAND WINE AND LIQUOR DENVER CO,False,True,-11.87,Discretionary,Alcohol,NaN,NaN,2018,9,10
4,USAA CC,2018-09-01,TOWN LIQUOR 0848700216503DENVER CO,False,True,-22.68,Discretionary,Alcohol,NaN,NaN,2018,9,1


In [34]:
num_months = float(len(discretionary_df.month.unique()))
discretionary_cat_df = discretionary_df.groupby(['category'])['Amount'].sum().reset_index()

for i, cat in enumerate(discretionary_cat_df.category.unique()):
    cat_spend_mean = float(discretionary_cat_df["Amount"][discretionary_cat_df["category"] == cat].sum()) / num_months
    print("{} : {}".format(cat, cat_spend_mean))

Alcohol : -121.37833333333333
Bar : -247.93499999999997
Bike : -52.028333333333336
Books : -16.09
Clothing : -211.74833333333333
Coffee : -10.753333333333332
Date : -104.34333333333332
Entertainment : -111.53500000000003
Experiences : -646.5566666666667
Gift : -73.27666666666666
Grocery : -519.9250000000001
Miscellaneous : -8.485000000000001
Personal Care : -48.87166666666666
Restaurant : -183.275
Snowboarding : -135.59666666666666
Tickets/Fees : -6.833333333333333
Tiny Home : -10.628333333333332
Transportation : -373.9800000000001
